In [1]:
from tools.ground_removal import *
from tools.voxelization import *
from sklearn.cluster import DBSCAN
import numpy as np
from mayavi import mlab
%load_ext autoreload
%autoreload 2

In [2]:
dynamic_points = get_dynamic_points(0.5, 15)

In [48]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], mode='point')
mlab.show()

In [3]:
clustering = DBSCAN(eps = 0.6, min_samples=8).fit(dynamic_points)
labels = (clustering.labels_ + 1) / (clustering.labels_.max() + 1)

In [4]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], labels, mode='cube',scale_mode = 'none', scale_factor=0.2)
mlab.show()

## TODO: dynamic points delat pomocne pole kde 0 znaci staticky bod, 1 znaci dynamicky a nedelat if x in voxels
firstly use clustering to separate noise from dynamic points - will still get some noise and dynamic objects will form separate cluster for each time frame, use clustering again to connect close clusters and eliminate clusters with no close neighbour cluster

In [5]:
filtered_dynamic_points = dynamic_points[labels > 0]
filtered_labels = labels[labels > 0]
print("dynamic points " + str(len(filtered_labels)))

dynamic points 1136


In [6]:
clustering2 = DBSCAN(eps = 3.0, min_samples=30).fit(filtered_dynamic_points)
labels2 = (clustering2.labels_ + 1) / (clustering2.labels_.max() + 1)
filtered_dynamic_points2 = filtered_dynamic_points[labels2 > 0]
labels2 = labels2[labels2>0]

In [7]:
mlab.points3d(filtered_dynamic_points2[:,0], filtered_dynamic_points2[:,1], filtered_dynamic_points2[:,2], labels2, mode='cube',scale_mode = 'none', scale_factor=0.3)
mlab.show()

In [8]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], mode='cube',scale_mode = 'none', scale_factor=0.1)
mlab.points3d(filtered_dynamic_points2[:,0], filtered_dynamic_points2[:,1], filtered_dynamic_points2[:,2], labels2, mode='cube',scale_mode = 'none', scale_factor=0.2)
mlab.show()

# TODO
leverage the fact that the static points (which still remains after clustering) do not change position during time

Ground truth dynamic points

In [9]:
pts, intens = get_frame(0)
dynamic_mask = get_dynamic_points_mask(0)
pts = pts[dynamic_mask]
print(dynamic_mask.sum())
#print(pts[dynamic_mask].shape)

for frame in range(1,15):
    p, i = get_frame(frame)
    m = get_dynamic_points_mask(frame)
    pts = np.concatenate((pts, p[m]))
    #intens = np.concatenate((intens, i))
    #dynamic_mask = np.concatenate((dynamic_mask, m))


330


In [14]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], mode='cube',scale_mode = 'none', scale_factor=0.1)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], mode='cube', scale_mode = 'none', scale_factor=0.1, color=(1,0,0))
mlab.show()

In [9]:
dynamic_mask.shape

(122829,)

In [10]:
pts.shape

(122829, 3)

In [105]:
file_name = "0" * int(6 - (len(str(1)))) + str(1)
labels = np.fromfile(PATH + "labels/" + file_name + ".label", dtype=np.uint32)

In [106]:
labels[dynamic_mask]

array([     1,  65788,  65788,  65788,  65788,  65788,  65788,  65788,
        65788,  65788,  65788,  65788,  65788,  65788,      1,  65788,
        65788,  65788,  65788,  65788,      1, 131324, 131324, 131324,
       131324, 131324,  65788,  65788,  65788,  65788,  65788,  65788,
        65788,  65788,  65788,  65788,  65788,  65788,  65788,  65788,
        65788,  65788,  65788,  65788,  65788,  65788,  65788,  65788,
        65788,  65788,  65788,  65788,  65788,  65788,  65788,  65788,
        65788, 327932, 327932, 327932, 131324, 131324, 131324, 131324,
       131324, 131324, 131324, 131324, 131324, 131324, 131324, 131324,
       131324, 131324, 131324, 131324, 131324, 262396, 131324, 131324,
       131324, 131324, 131324, 131324, 131324, 131324, 131324, 131324,
       131324, 131324, 131324, 131324, 262396, 131324, 131324, 131324,
       131324, 131324, 131324, 131324, 131324, 131324, 131324, 131324,
       131324, 131324, 131324, 131324, 131324, 131324, 131324, 131324,
      